In [ ]:
import devito
import examples.seismic
import hdf5eis
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import scipy.signal

FLOAT_DTYPE = np.float32

def plot_field(field, src_coords=None, it=None, z_positive_down=False, amax=None, cmap=plt.get_cmap('seismic')):
    
    x0, y0, z0 = field.grid.origin
    nx, ny, nz = field.grid.shape
    dx, dy, dz = field.grid.spacing
    xx, yy, zz = np.meshgrid(
        x0 + np.arange(nx)*dx,
        y0 + np.arange(ny)*dy,
        z0 + np.arange(nz)*dz,
        indexing='ij'
    )

    if src_coords is None:
        ix = nx//2
        iy = ny//2
        iz = nz//2
        src_coords = np.array([x0+(ix-1)*dx, y0+(iy-1)*dy, z0+(iz-1)*dz])
    else:
        ix, iy, iz = np.round(
            (src_coords - field.grid.origin) / field.grid.spacing
        ).astype(int)
    data = field.data if it is None else field.data[it]

    x_range = dx*(nx-1)
    y_range = dy*(ny-1)
    z_range = dz*(nz-1)
    w = [x_range/(x_range+z_range), z_range/(x_range+z_range)]
    h = [y_range/(y_range+z_range), z_range/(y_range+z_range)]
    gs = mpl.gridspec.GridSpec(2, 2, width_ratios=w, height_ratios=h)
    fig = plt.figure()
    ax_xy = fig.add_subplot(gs[0, 0], aspect=1, anchor='SE')
    ax_yz = fig.add_subplot(gs[0, 1], aspect=1, sharey=ax_xy, anchor='SW')
    ax_xz = fig.add_subplot(gs[1, 0], aspect=1, sharex=ax_xy, anchor='NE')
    plt.subplots_adjust(hspace=0, wspace=0)


    if amax is None:
        amax = np.max([
            np.max(np.abs(data[ix])),
            np.max(np.abs(data[:, iy])),
            np.max(np.abs(data[:, :, iz])),
        ])
    kwargs = dict(
        vmin=-amax,
        vmax=amax,
        cmap=cmap
    )
    ax_xy.pcolormesh(
        xx[:, :, iz],
        yy[:, :, iz],
        data[:, :, iz],
        **kwargs
    )
    ax_xz.pcolormesh(
        xx[:, iy],
        zz[:, iy],
        data[:, iy],
        **kwargs
    )
    ax_yz.pcolormesh(
        zz[ix],
        yy[ix],
        data[ix],
        **kwargs
    )
    for ax, vline, hline in zip(
        (ax_xy, ax_xz, ax_yz),
        src_coords[[0, 0, 2]],
        src_coords[[1, 2, 1]]
    ):
        ax.axvline(vline, color='k', linewidth=1)
        ax.axhline(hline, color='k', linewidth=1)
        
    ax_xy.xaxis.tick_top()
    ax_yz.xaxis.tick_top()
    ax_yz.yaxis.tick_right()
    if z_positive_down is True:
        ax_xz.invert_yaxis()
    else:
        ax_yz.invert_xaxis()

    return dict(
        ax_xy=ax_xy,
        ax_xz=ax_xz,
        ax_yz=ax_yz
    )

## Define the physical model domain

In [ ]:
network = pd.read_hdf('../data/network.hdf5')
network = network.drop_duplicates(subset=['network', 'station'])
network['z'] = network['elevation'] - network['depth']

In [ ]:
f0          = 32   # Peak frequency (in Hz) of Ricker wavelet source
v_min       = 1000 # Minimum wavespeed in m/s
v_max       = 1000
f_max       = f0
h_max       = v_min / f_max / 10
nbl         = 20          # Number of absorbing boundary layers.
bcs         = "damp"      # Abosrbing boundary type.
space_order = 8
time_order  = 2
dt          = 0.49 * h_max / v_max

# x_min, y_min, z_min = 0, 0, 0
# x_max, y_max, z_max = 2e3, 2e3, 2e3
x_min, y_min = network[['easting', 'northing']].min()
x_max, y_max, z_max = network[['easting', 'northing', 'z']].max()
z_min = z_max - 0.5*np.max([(x_max - x_min), (y_max-y_min)])

dx, dy, dz          = h_max, h_max, h_max
x_pad, y_pad, z_pad = 40, 40, 40
nx, ny, nz          = (np.ceil(
    [x_max-x_min, y_max-y_min, z_max-z_min]
) / np.array([dx, dy, dz])).astype(int) + 1

# Grid spacing in m.
spacing = (h_max, h_max, h_max)

# Number of grid points (nx, nz).
shape = (
    nx+2*x_pad, 
    ny+2*y_pad, 
    nz+2*z_pad
)

# What is the location of the top left corner.
origin = (
    x_min-x_pad*dx, 
    y_min-y_pad*dy, 
    z_min-z_pad*dz
)



v = v_min * 1e-3 * np.ones(shape, dtype=FLOAT_DTYPE)
# v += np.abs(scipy.ndimage.gaussian_filter(np.random.randn(*shape), 2))

print('', shape, '\n', spacing, '\n', origin, '\n', nbl)

In [ ]:
model = examples.seismic.Model(
    vp=v,
    origin=origin,
    shape=shape,
    spacing=spacing,
    space_order=space_order,
    nbl=nbl, 
    bcs=bcs
)
res = plot_field(
    model.damp,
    z_positive_down=False
);
res['ax_xy'].scatter(
    network['easting'], 
    network['northing'],
    s=2,
    linewidth=0
)

# Define the source

In [ ]:
t0         = 0.                                # Simulation start time (in ms).
tn         = 1000.                              # Simulation duration (in ms).
dt         = model.critical_dt                 # Simulation time step from model grid spacing.
# src_coords = np.array([x_min+x_max, y_min+y_max, z_min+z_max])/2 # Source location coordinates
src_coords = np.mean([[x_min, x_max], [y_min, y_max], [z_min, z_max]], axis=1)

time_range = examples.seismic.TimeAxis(start=t0, stop=tn, step=dt)
src = examples.seismic.RickerSource(
    name='s', 
    grid=model.grid, 
    f0=f0*1e-3,
    coordinates=src_coords,
    time_range=time_range,
    t0=t0+100
)
src.show()

In [ ]:
src_coords[0], src_coords[1], src_coords[2]

# Define the receiver geometry

In [ ]:
#NBVAL_IGNORE_OUTPUT
from devito import ConditionalDimension, TimeFunction, Eq, solve, Operator

nt = time_range.num

nsnaps = 16            # desired number of equally spaced snaps
factor = round(time_range.num / nsnaps)  # subsequent calculated factor

time_subsampled = ConditionalDimension(
    't_sub', 
    parent=model.grid.time_dim, 
    factor=factor
)
usave = TimeFunction(
    name='usave', 
    grid=model.grid, 
    time_order=2, 
    space_order=2,
    save=(time_range.num + factor - 1) // factor, 
    time_dim=time_subsampled
)

u = TimeFunction(
    name='u', 
    grid=model.grid, 
    time_order=time_order, 
    space_order=space_order
)

pde = model.m * u.dt2 - u.laplace + model.damp * u.dt
stencil = Eq(u.forward, solve(pde, u.forward))
src_term = src.inject(
    field=u.forward,
    expr=src * dt**2 / model.m,
    # offset=model.nbl
)

# Create symbol for 101 receivers
rec = examples.seismic.Receiver(
    name='r', 
    grid=model.grid,
    coordinates=network[['easting', 'northing', 'z']].values,
    time_range=time_range
)
rec_term = rec.interpolate(
    expr=u, 
    # offset=model.nbl
)

op = Operator([stencil] + src_term + rec_term + [Eq(usave, u)],
               subs=model.spacing_map)  # operator with snapshots

op(time=nt-1, dt=model.critical_dt)

In [ ]:
amax = np.max(np.abs(rec.data))

plt.close('all')
fig, ax = plt.subplots()
ax.pcolorfast(
    rec.data, 
    cmap=plt.get_cmap('seismic'), 
    vmin=-amax,
    vmax=amax
)
ax.invert_yaxis()
# ax.p(rec.time_values, rec.data[:, 500])

In [ ]:
plot_field(usave, src_coords=src_coords, it=9, z_positive_down=False)

In [ ]:
network = pd.read_hdf('../data/network.hdf5')
network = network.drop_duplicates(subset=['network', 'station'])
network['z'] = network['elevation'] - network['depth']

with hdf5eis.File(f'../data/test-{f0}Hz.hdf5', mode='w', overwrite=True) as out_file:
    out_file.timeseries.add(rec.data.T, 0, 1/(time_range.step*1e-3), tag='synthetic/raw')
    out_file.metadata.add_table(network, key='network')

In [ ]:
tag = 'synthetic/raw'
with hdf5eis.File(f'../data/test-{f0}Hz.hdf5', mode='r') as in_file:
    index = in_file.timeseries.index
    super_gather = in_file.timeseries[tag, ..., 0:1e9]
    
gather = super_gather[tag][0]
    
plt.close('all')
fig, ax = plt.subplots()
ax.plot(np.arange(len(gather.data[0]))/gather.sampling_rate, gather.data[0])

In [ ]:
index

In [ ]:
151059 / 377647.5

In [ ]:
gather.data[0]

In [ ]:
new_sampling_rate = 500

X, t = scipy.signal.resample(rec.data, int(new_sampling_rate *(tn-t0)*1e-3), t=time_range.time_values*1e-3)

In [ ]:
rec.data.shape

In [ ]:
1 / rec.time_range.step

In [ ]:
model.grid.shape

In [ ]:
model.grid.origin

In [ ]:
src_coords

In [ ]:
network = network.drop_duplicates(subset=['network', 'station'])

# Now do reverse time migration

In [ ]:
IRXS = range(0, rec.npoint, 32)

In [ ]:
Ws         = dict() # Reverse-time migrated wavefield for each station
kernels    = dict() # Computational kernel for each wavefield extrapolation problem

for irx in IRXS:
    handle = f"{irx:03d}"
    W = devito.TimeFunction(
        name=f"W_{handle}", 
        grid=model.grid, 
        time_order=2,
        space_order=2
    )
    Ws[irx] = W
    
    pde = model.m * W.dt2 - W.laplace + model.damp * W.dt 
    
    stencil = devito.Eq(W.forward, devito.solve(pde, W.forward))
    
    rsrc = examples.seismic.PointSource(
        name=f"X_{handle}",
        grid=model.grid,
        time_range=time_range,
        npoint=1,
        coordinates=rec.coordinates_data[irx],
        space_order=2,
        time_order=2
    )
    # rsrc.data[:] = rec_a.data[-1::-1, [irx]]
    # rsrc.data[:] = rec_b.data[-1::-1, [irx]]
    rsrc.data[:] = (rec_b.data[-1::-1, [irx]]+4*np.random.randn(*rsrc.data.shape)) - (rec_a.data[-1::-1, [irx]]+4*np.random.randn(*rsrc.data.shape))
    # rsrc.data[:] += 4*np.random.randn(*rsrc.data.shape)
    rsrc_term    = rsrc.inject(field=W.forward, expr=rsrc * dt**2 / model.m)
    
    kernels[irx] = [stencil, rsrc_term]
    
image = devito.Function(
    name=f"Im", 
    grid=model.grid
)

In [ ]:
%%time
imaging_kernel = devito.Eq(image, (image + np.prod([w for w in Ws.values()])))

op = devito.Operator(
    [eq for kernel in kernels.values() for eq in kernel] + [imaging_kernel], 
    subs=model.spacing_map
)
op(dt=model.critical_dt);

In [ ]:
image_b = image.data.copy()

In [ ]:
plt.close("all")
# ax, kw = plot_field((image_a.T+image_b.T)/2, model.grid, nbl=model.nbl)
# ax0, kw = plot_field(image_a.T, model.grid, nbl=model.nbl)
# ax1, kw = plot_field(image_b.T, model.grid, nbl=model.nbl)
ax, kw = plot_field(image.data.T, model.grid, nbl=model.nbl)

for src in (src_a, src_b):
    ax.scatter(
        src.coordinates_data[:, 0],
        src.coordinates_data[:, 1],
        edgecolor="k"
    )
# plot_field(Ws[irx].data[-1].T, model.grid);

In [ ]:
plt.close("all")
ax, kw = plot_field(image.data.T, model.grid, nbl=model.nbl)
ax.scatter(
    src.coordinates_data[:, 0],
    src.coordinates_data[:, 1],
    color="tab:red",
    edgecolor="k"
)